# Read All Dataset CSV

In [29]:
import os
import csv
import pandas as pd
import numpy as np

In [30]:
from sklearn.preprocessing import LabelEncoder

dataset_names=[]
X_trains=[]
y_trains=[]
X_tests=[]
for folder_name in os.listdir("./Competition_data"):
    # print(folder_name)
    
    x_train = pd.read_csv(f"./Competition_data/{folder_name}/X_train.csv",header=0)
    y_train = pd.read_csv(f"./Competition_data/{folder_name}/y_train.csv",header=0)
    x_test = pd.read_csv(f"./Competition_data/{folder_name}/X_test.csv",header=0)  
    
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()
    categorical_columns = x_train.columns[7:20]
    # Apply LabelEncoder to each categorical column
    X_train_encoded = x_train.copy()
    X_test_encoded = x_test.copy()
    for col in categorical_columns:
        # Fit the label encoder on the combined data of train and test to avoid unseen labels
        combined_data = pd.concat([x_train[col], x_test[col]], axis=0)
        label_encoder.fit(combined_data)

        X_train_encoded[col] = label_encoder.transform(x_train[col])
        X_test_encoded[col] = label_encoder.transform(x_test[col])

    dataset_names.append(folder_name)
    X_trains.append(X_train_encoded)
    y_trains.append(y_train)
    X_tests.append(X_test_encoded)

        

## Data Preprocessing & Feature Engineering

In [31]:
## your code here
# for x_train, x_test in zip(X_trains, X_tests):


    


## train test split & build Model
You can select an appropriate model and perform corresponding hyperparameter tuning.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score

使用stack處理

In [ ]:
import xgboost as xgb

y_predicts=[]
for i in range(len(dataset_names)):
    X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_trains[i], y_trains[i], test_size=0.2, random_state=42)
    
    # 準備基模型
    base_models = [  
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),
        ('xgb', xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42))
    ]

    # 準備次級模型
    meta_model = LogisticRegression()

    # 初始化Stacking Classifier
    stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=10)

    # 訓練Stacking Classifier
    stacking_model.fit(X_train_part, y_train_part)

    # 使用驗證集進行預測
    y_pred = stacking_model.predict(X_valid)
    y_pred_proba = stacking_model.predict_proba(X_valid)[:, 1] 

    # 評估模型
    accuracy = accuracy_score(y_valid, y_pred)
    # f1 = f1_score(y_valid, y_pred, average='weighted')
    # precision = precision_score(y_valid, y_pred, average='weighted')

    # print(f"Accuracy: {accuracy:.2f}")
    # print(f"F1 Score: {f1:.2f}")
    # print(f"Precision: {precision:.2f}")
    print(f"{i}times finish")
    print(f"AUC: {accuracy:.2f}")

    # 使用訓練好的Stacking模型來預測X_test並儲存結果
    y_test_pred = stacking_model.predict_proba(X_tests[i])[:, 1] 
    df = pd.DataFrame(y_test_pred, columns=['y_predict_proba'])
    y_predicts.append(df)


on 0: 0times finish
on 0: AUC: 0.80


on 0: c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
        y = column_or_1d(y, warn=True)
      c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
        y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


on 1: 1times finish
on 1: AUC: 0.71


on 1: c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
        y = column_or_1d(y, warn=True)
      c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
        y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


on 2: 2times finish
on 2: AUC: 0.58


...

on 47: 47times finish
on 47: AUC: 0.86


on 47: c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
         y = column_or_1d(y, warn=True)
       c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
         y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


on 48: 48times finish
on 48: AUC: 0.79


on 48: c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
         y = column_or_1d(y, warn=True)
       c:\Users\user\OneDrive\桌面\大四上\A-general-model-for-Binary-Classification\.venv\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
         y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


|████████████████████████████████████████| 49/49 [100%] in 3:36.0 (0.23/s) 


## Save result

In [34]:
for idx,dataset_name in enumerate(dataset_names):
    df=y_predicts[idx]
    df.to_csv(f'./Competition_data/{dataset_name}/y_predict.csv', index=False,header=True)